In [6]:
import requests
from bs4 import BeautifulSoup
import json
import re
import os
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver as sw
from webdriver_manager.chrome import ChromeDriverManager

file = os.path.dirname(os.path.realpath("__file__"))

if os.path.exists(f'{file}/Amazon_image') == False: #檢查路徑是否存在
    os.mkdir(f'{file}/Amazon_image') #創建目錄
data = []
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36"}

#<-------------selenium設定-----------
options = Options()
options.add_argument("--disable-notifications") #阻止google視窗通知跳出
options.add_argument('--headless') # 啟動無頭模式
options.add_argument('--disable-gpu') # windows必須加入此行
options.experimental_options['prefs']={'profile.default_content_settings':{'images':2},'profile.managed_default_content_settings':{"images":2}} #不顯示圖片
#------------------------->
url_class =['https://www.amazon.com/gcx/Tools-&-Home-Improvement/gfhz/events?canBeGiftWrapped=false&categoryId=GRFWW21-THI&isLimitedTimeOffer=false&isPrime=false'
 ,'https://www.amazon.com/gcx/College-Electronics/gfhz/events?canBeGiftWrapped=false&categoryId=OTC21-College-Tech&isLimitedTimeOffer=false&isPrime=false&scrollState=eyJpdGVtSW5kZXgiOjAsInNjcm9sbE9mZnNldCI6MTEwLjE0MDYyNX0%3D&sectionManagerState=eyJzZWN0aW9uVHlwZUVuZEluZGV4Ijp7ImFtYWJvdCI6MX19',]
for u in url_class:
    driver = sw.Chrome(ChromeDriverManager().install(), chrome_options=options)
    driver.get(u)
    top = driver.find_element_by_xpath('//*[@id="navBackToTop"]/div/span')
    ActionChains(driver).move_to_element(top).perform() #視窗往下移

    soup = BeautifulSoup(driver.page_source , 'html.parser')
    big_tag = [] #大分類
    big_tag_url = [] #大分類url
    for r in range(0 , 10):
        for i in soup.select(f'div#sobe_d_aw_{r}-container ol li'):
            tag_url = 'https://www.amazon.com'+i.select('div.sl-sobe-carousel-sub-card-footer a')[0]['href']
            tag_name = i.select('div.sl-sobe-carousel-sub-card-footer a')[0].text.strip()
            big_tag.append(tag_name)
            big_tag_url.append(tag_url)
            print(tag_name)

    driver.close()
    a=0
    print(big_tag_url) #喚醒python 
    #進入爬取頁面
    for v in range(0 , len(big_tag)):
        driver = sw.Chrome(ChromeDriverManager().install(), chrome_options=options)
        print(big_tag_url[v])
        driver.get(big_tag_url[v])
        top = driver.find_element_by_xpath('//*[@id="navBackToTop"]/div/span')
        ActionChains(driver).move_to_element(top).perform()
        soup = BeautifulSoup(driver.page_source , 'html.parser')
        categorys = soup.select('div.sc-1wcpl6x-0.cIlFpO ul li')
        medium_tag = [] #中分類
        tag = set() #中分類_tag
        for category in categorys:
            tag.add(category.select('button')[0]['data-test'].split(':')[1])
            medium_tag.append(category.select('button')[0]['data-test'].split(':')[2])
        driver.close()
        count = 1000
        tag = list(tag)

        for mt in medium_tag:
            url = f'https://www.amazon.com/gcx/-/gfhz/api/scroll?canBeGiftWrapped=false&categoryId={tag[0]}&count={count}&isLimitedTimeOffer=false&isPrime=false&offset=0&priceFrom&priceTo&searchBlob&subcategoryIds={tag[0]}%3A{mt}'
            res = requests.get(url=url , headers=headers)
            json_data = json.loads(res.text)
            for i in json_data['asins']:
                data_dict = dict()
                product_title = i['title']#品名
                product_price = i['price'] #價格
                product_comments = i['reviewCount'] #評論數
                product_star = i['starRating'] #評分
                product_image =requests.get(i['displayLargeImageURL'])
                jpg_title =re.sub('\*|\\|-|\?|"|<|>|\||/|\\\\|:','_', product_title)
#                 time.sleep(1)
#                 if os.path.isfile(f'{file}/Amazon_image/{jpg_title}.jpg') == False: 
#                     with open(f'{file}/Amazon_image/{jpg_title}.jpg' , 'wb') as file_obj :
#                         file_obj.write(product_image.content)
#                 else :
#                     print('檔案已存在')

                data_dict['商品名'] = product_title
                data_dict['價格'] = product_price
                data_dict['大分類'] = big_tag[v]
                data_dict['中分類'] = mt
                data_dict['評論數'] = product_comments
                data_dict['評分'] = product_star
                data_dict['圖片URL'] = i['displayLargeImageURL']
#                 data_dict['圖片名'] = f'{jpg_title}.jpg'
#                 data_dict['路徑'] = f'./Amazon_image/{jpg_title}.jpg'
                data.append(data_dict)
                a = a+1
                print(a)
                

# print(data)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\Tibame_25\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache
<ipython-input-6-ce2f3e688fe9>:29: DeprecationWarning: use options instead of chrome_options
  driver = sw.Chrome(ChromeDriverManager().install(), chrome_options=options)


====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\Tibame_25\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


[]




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\Tibame_25\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


Laptops & PCs
Desk and Study Supplies
On the Go
Smart Room
Headphones
Tech Accessories
Gaming & Entertainment
Room Electronics
Fitness Tech
Content Creators
Amazon Renewed
['https://www.amazon.com/gcx/Laptops-&-PCs/gfhz/events/ref=sp21sobe_1?pf_rd_r=T9632QXP39RA3EV16K7E&pf_rd_p=843364b4-92ef-45b1-af52-5610afccdf43&pf_rd_m=ATVPDKIKX0DER&pf_rd_s=top-slot-7&pf_rd_t=&pf_rd_i=&categoryId=OTC21-Laptops', 'https://www.amazon.com/gcx/Desk-and-Study-Supplies/gfhz/events/ref=sp21sobe_2?pf_rd_r=T9632QXP39RA3EV16K7E&pf_rd_p=843364b4-92ef-45b1-af52-5610afccdf43&pf_rd_m=ATVPDKIKX0DER&pf_rd_s=top-slot-7&pf_rd_t=&pf_rd_i=&categoryId=OTC21-Desk', 'https://www.amazon.com/gcx/On-the-Go/gfhz/events/ref=sp21sobe_3?pf_rd_r=T9632QXP39RA3EV16K7E&pf_rd_p=843364b4-92ef-45b1-af52-5610afccdf43&pf_rd_m=ATVPDKIKX0DER&pf_rd_s=top-slot-7&pf_rd_t=&pf_rd_i=&categoryId=OTC21-Go', 'https://www.amazon.com/gcx/Smart-Dorm/gfhz/events/ref=sp21sobe_4?pf_rd_r=T9632QXP39RA3EV16K7E&pf_rd_p=843364b4-92ef-45b1-af52-5610afccdf43&pf

<ipython-input-6-ce2f3e688fe9>:50: DeprecationWarning: use options instead of chrome_options
  driver = sw.Chrome(ChromeDriverManager().install(), chrome_options=options)


...

841
842
843
844
845
846
847


Get LATEST driver version for 91.0.4472
Driver [C:\Users\Tibame_25\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


https://www.amazon.com/gcx/Content-Creators/gfhz/events/ref=sp21sobe_5?pf_rd_r=T9632QXP39RA3EV16K7E&pf_rd_p=5266e1e2-80a1-40ac-8986-f899b9c95d5f&pf_rd_m=ATVPDKIKX0DER&pf_rd_s=top-slot-8&pf_rd_t=&pf_rd_i=&categoryId=OTC21-Content
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\Tibame_25\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


952
953
https://www.amazon.com/gcx/Renewed-College-Essentials/gfhz/events/ref=sp21sobe_6?pf_rd_r=T9632QXP39RA3EV16K7E&pf_rd_p=5266e1e2-80a1-40ac-8986-f899b9c95d5f&pf_rd_m=ATVPDKIKX0DER&pf_rd_s=top-slot-8&pf_rd_t=&pf_rd_i=&categoryId=amazon-renewed-otc
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112

In [8]:
import pandas as pd
df = pd.DataFrame(data = data)
df.to_csv(f'{file}/Amazon_product.csv' , index=False , encoding='utf-8-sig')

df.head(200)

,商品名,價格,大分類,中分類,評論數,評分,圖片URL
0,Logitech Combo Touch iPad Pro 12.9-inch (5th G...,$229.99,Laptops & PCs,Accessories,36,4.6,https://m.media-amazon.com/images/I/41TJe4tNne...
1,OtterBox DEFENDER SERIES Case for iPad 8th & 7...,$76.49,Laptops & PCs,Accessories,"5,571",4.8,https://m.media-amazon.com/images/I/41zYU9lCoa...
2,Amazon Basics Adjustable Tablet Holder Stand -...,$10.99,Laptops & PCs,Accessories,"21,502",4.6,https://m.media-amazon.com/images/I/31lR838Rjt...
3,Amazon Basics Premium Single Monitor Stand - L...,$116.00,Laptops & PCs,Accessories,"5,845",4.6,https://m.media-amazon.com/images/I/41jc7zYC4v...
4,"HP Desktop PC, Intel Pentium Gold G6400 Proces...",$499.99,Laptops & PCs,Desktop,106,4.2,https://m.media-amazon.com/images/I/41bFRiEw1b...
...,...,...,...,...,...,...,...
195,Samsung Galaxy S21 5G | Factory Unlocked Andro...,$699.99,On the Go,Cell-Phones-and-Accessories,237,4.5,https://m.media-amazon.com/images/I/41nvTAT6Ep...
196,Samsung Galaxy S21 5G | Factory Unlocked Andro...,$699.99,On the Go,Cell-Phones-and-Accessories,175,4.7,https://m.media-amazon.com/images/I/410yyetaNW...
197,"OnePlus Nord N10 5G Unlocked Smartphone, Midni...",$299.99,On the Go,Cell-Phones-and-Accessories,552,4.3,https://m.media-amazon.com/images/I/3121+oL3B3...
198,Motorola One 5G Ace | 2021 | 2-Day battery | U...,$379.99,On the Go,Cell-Phones-and-Accessories,498,4.3,https://m.media-amazon.com/images/I/41FdYLBo+N...


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1150 entries, 0 to 1149
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   商品名     1150 non-null   object 
 1   價格      1150 non-null   object 
 2   大分類     1150 non-null   object 
 3   中分類     1150 non-null   object 
 4   評論數     1150 non-null   object 
 5   評分      1150 non-null   float64
 6   圖片URL   1150 non-null   object 
dtypes: float64(1), object(6)
memory usage: 63.0+ KB
